# Detect size of object

### “pixels per enhed” forholdet
For at beregne størrelsen af et objekt i et billedet, kræves det at man først udførere en "kalibrering" vha. et reference objekt.
Reference objektet bør besside 2 vigtige egenskaber:

1. Man skal kende objektets dimensioner (højde/bredde)
2. Man skal nemt kunne finde/genkende objektet i billedet, endeten baseret på en fast placering i billedet eller vha. udseende (som fx. farve eller form).



I denne opgave har vi valgt at anvende en dansk 20 krone som vores reference objekt. Vi har, for at finde reference punktet, valgt at 20 kronen altid vil være det objekt i billedet der er længest mod venstre.

Den danske 20 krone måler 27mm i diameteren.

Da vi nu kan garantere at 20 kronen altid vil være objektet længest mod venstre, kan vi sortere vores objekt konturer fra højre mod venstre, detektere 20 kronen (som altid vil være den 1. kontor i den sortede kontur liste), og anvende denne til at udregne vores "pixels_per_metric" som er defineret ved:

pixels_per_metric = object_width / know_width

En 20 krone har en known_width på 27 mm. Hvis man da antager at man har udregnet en object_width (målt i pixels) til at være 150 pixels bred, vil formlen se ud på følgende vis:


pixels_per_metric = 150px / 27mm = 5.5555px


Hvilket betyder at der ca. er 5.5 pixels på 150px.

Med viden om dette forhold kan man udregne størrelsen af objekterne i resten af billedet.

In [2]:
# import the necessary packages
from scipy.spatial import distance as dist
from imutils import perspective
from imutils import contours
import numpy as np
import argparse
import imutils
import cv2

def midpoint(ptA, ptB):
    return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)

Ovenfor defineres en helper method kaldt "midpoint", der som navnet antyder, kan anvendes til at beregne midpunktet mellem 2 sæt (x, y)-koordinater.

In [6]:
def preprocess_image(path_to_image):
    # load the image, convert it to grayscale, and blur it slightly
    image = cv2.imread(path_to_image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)

    # perform edge detection, then perform a dilation + erosion to
    # close gaps in between object edges
    edged = cv2.Canny(gray, 50, 100)
    edged = cv2.dilate(edged, None, iterations=1)
    edged = cv2.erode(edged, None, iterations=1)

    # find contours in the edge map
    cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if imutils.is_cv2() else cnts[1]

    # sort the contours from left-to-right and initialize the
    # 'pixels per metric' calibration variable
    (cnts, _) = contours.sort_contours(cnts)
    pixelsPerMetric = None

**Lines 22-24** load our image from disk, convert it to grayscale, and then smooth it using a Gaussian filter. We then perform edge detection along with a dilation + erosion to close any gaps in between edges in the edge map (Lines 28-30).

**Lines 33-35** find contours (i.e., the outlines) that correspond to the objects in our edge map.

These contours are then sorted from left-to-right (allowing us to extract our reference object) on **Line 39**. 

We also initialize our pixelsPerMetric  value on **Line 40**.


The next step is to examine each of the contours:

In [7]:
def compute_size_of_contours(cnt):
    # loop over the contours individually
    for c in cnts:
        # if the contour is not sufficiently large, ignore it
        if cv2.contourArea(c) < 100:
            continue

        # compute the rotated bounding box of the contour
        orig = image.copy()
        box = cv2.minAreaRect(c)
        box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
        box = np.array(box, dtype="int")

        # order the points in the contour such that they appear
        # in top-left, top-right, bottom-right, and bottom-left
        # order, then draw the outline of the rotated bounding
        # box
        box = perspective.order_points(box)
        cv2.drawContours(orig, [box.astype("int")], -1, (0, 255, 0), 2)

        # loop over the original points and draw them
        for (x, y) in box:
            cv2.circle(orig, (int(x), int(y)), 5, (0, 0, 255), -1)

On Line 43 we start looping over each of the individual contours. If the contour is not sufficiently large, we discard the region, presuming it to be noise left over from the edge detection process (Lines 45 and 46).

Provided that the contour region is large enough, we compute the rotated bounding box of the image on Lines 50-52, taking special care to use the cv2.cv.BoxPoints  function for OpenCV 2.4 and the cv2.boxPoints  method for OpenCV 3.

We then arrange our rotated bounding box  coordinates in top-left, top-right, bottom-right, and bottom-left order, as discussed in last week’s blog post (Line 58).

Lastly, Lines 59-63 draw the outline of the object in green, followed by drawing the vertices of the bounding box rectangle in as small, red circles.

Now that we have our bounding box ordered, we can compute a series of midpoints:

In [ ]:
def compute_midpoints():
    # unpack the ordered bounding box, then compute the midpoint
    # between the top-left and top-right coordinates, followed by
    # the midpoint between bottom-left and bottom-right coordinates
    (tl, tr, br, bl) = box
    (tltrX, tltrY) = midpoint(tl, tr)
    (blbrX, blbrY) = midpoint(bl, br)

    # compute the midpoint between the top-left and top-right points,
    # followed by the midpoint between the top-righ and bottom-right
    (tlblX, tlblY) = midpoint(tl, bl)
    (trbrX, trbrY) = midpoint(tr, br)

    # draw the midpoints on the image
    cv2.circle(orig, (int(tltrX), int(tltrY)), 5, (255, 0, 0), -1)
    cv2.circle(orig, (int(blbrX), int(blbrY)), 5, (255, 0, 0), -1)
    cv2.circle(orig, (int(tlblX), int(tlblY)), 5, (255, 0, 0), -1)
    cv2.circle(orig, (int(trbrX), int(trbrY)), 5, (255, 0, 0), -1)

    # draw lines between the midpoints
    cv2.line(orig, (int(tltrX), int(tltrY)), (int(blbrX), int(blbrY)),
        (255, 0, 255), 2)
    cv2.line(orig, (int(tlblX), int(tlblY)), (int(trbrX), int(trbrY)),
        (255, 0, 255), 2)